In [ ]:
%pip install transformers==4.40.1 accelerate==0.29.3 bitsandbytes==0.43.1 datasets==2.19.0 vllm==0.4.1 openai==1.25.1 -qqq
%pip install numpy==1.24.4
%pip install triton==2.1.0
%pip install openai==1.25.2 httpx==0.27.0 pydantic==2.6.4

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached triton-2.1.0-0-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.3 kB)
Using cached triton-2.1.0-0-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (89.2 MB)
  Attempting uninstall: triton
    Found existing installation: triton 2.2.0
    Uninstalling triton-2.2.0:
      Successfully uninstalled triton-2.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.17.1+cu121 requires pillow!=8.3.*,>=5.3.0, which is not installed.
torch 2.2.1+cu121 requires triton==2.2.0; platform_system == "Linux" and platform_machine == "x86_64" and python_version < "3.12", but you have triton 2.1.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip show triton

Name: triton
Version: 2.2.0
Summary: A language and compiler for custom Deep Learning operations
Home-page: https://github.com/openai/triton/
Author: Philippe Tillet
Author-email: phil@openai.com
License: 
Location: /home/young/.local/lib/python3.10/site-packages
Requires: filelock
Required-by: torch
Note: you may need to restart the kernel to use updated packages.


# 8.4절 실습: LLM 서빙 프레임워크

## 예제 8.1. 실습에 사용할 데이터셋 준비

In [1]:
import torch
from datasets import load_dataset

def make_prompt(ddl, question, query=''):
    prompt = f"""당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question을 해결할 수 있는 SQL 쿼리를 생성하세요.

### DDL:
{ddl}

### Question:
{question}

### SQL:
{query}"""
    return prompt

dataset = load_dataset("shangrilar/ko_text2sql", "origin")['test']  # 데이터셋 다운로드
dataset = dataset.to_pandas()

for idx, row in dataset.iterrows():
  prompt = make_prompt(row['context'], row['question']) # 프롬프트 형식으로 변환
  dataset.loc[idx, 'prompt'] = prompt

/home/young/anaconda3/envs/llm121/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 예제 8.2. 모델과 토크나이저를 불러와 추론 파이프라인 준비

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_id = "shangrilar/yi-ko-6b-text2sql"
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(model_id)
hf_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer) # 추론 파이프라인 구축

/home/young/anaconda3/envs/llm121/lib/python3.10/site-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|██████████| 3/3 [00:12<00:00,  4.10s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## 예제 8.3. 배치 크기에 따른 추론 시간 확인

In [3]:
import time
for batch_size in [1, 2, 4, 8, 16, 32]: # 배치 크기에 따른 추론 시간 확인
  start_time = time.time()
  hf_pipeline(dataset['prompt'].tolist(), max_new_tokens=128, batch_size=batch_size)
  print(f'{batch_size}: {time.time() - start_time}')

1: 155.32553553581238
2: 278.6676232814789
4: 186.9577476978302
8: 133.37742829322815
16: 98.06397271156311
32: 86.38982701301575


## 예제 8.4. vLLM 모델 불러오기

### 안내
모델을 여러 번 GPU에 올리기 때문에 CUDA out of memory 에러가 발생할 수 있습니다. 그런 경우 구글 코랩의 런타임 > 세션 다시 시작 후 예제 코드를 실행해주세요.
예제 실행에 데이터셋이 필요한 경우 예제 8.1의 코드를 실행해주세요.

In [3]:
import torch
from vllm import LLM, SamplingParams    # vllm 프레임워크 사용

model_id = "shangrilar/yi-ko-6b-text2sql"
llm = LLM(model=model_id, dtype=torch.float16, max_model_len=1024)

2025-04-20 21:36:45,119	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
/home/young/anaconda3/envs/llm121/lib/python3.10/site-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


INFO 04-20 21:36:45 llm_engine.py:98] Initializing an LLM engine (v0.4.1) with config: model='shangrilar/yi-ko-6b-text2sql', speculative_config=None, tokenizer='shangrilar/yi-ko-6b-text2sql', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=1024, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 04-20 21:36:45 utils.py:608] Found nccl from library /home/young/.config/vllm/nccl/cu12/libnccl.so.2.18.1
WARNING 04-20 21:36:45 utils.py:414] Using 'pin_memory=False' as WSL is detected. This may slow down the performance.
INFO 04-20 21:36:45 selector.py:77] Cannot use FlashAttention backend because the flash_attn package is not found. Please install it for better performance.
INFO 04-20 21:36:45 selector.py:33] Using XFormers backend.
INFO 04-20 21:36:47 weight_utils.py:193] Using model weights format ['*.safetensors']
INFO 04-20 21:36:57 model_runner.py:173] Loading model weights took 11.5127 GB
INFO 04-20 21:37:34 gpu_executor.py:119] # GPU blocks: 3886, # CPU blocks: 4096
INFO 04-20 21:37:34 model_runner.py:976] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 04-20 21:37:34 model_runner.py:980] CUDA graphs can take additi

## 예제 8.5. vLLM을 활용한 오프라인 추론 시간 측정

In [4]:
import time

for max_num_seqs in [1, 2, 4, 8, 16, 32]:
  start_time = time.time()
  llm.llm_engine.scheduler_config.max_num_seqs = max_num_seqs
  sampling_params = SamplingParams(temperature=1, top_p=1, max_tokens=128)
  outputs = llm.generate(dataset['prompt'].tolist(), sampling_params)
  print(f'{max_num_seqs}: {time.time() - start_time}')

Processed prompts: 100%|██████████| 112/112 [18:25<00:00,  9.87s/it]


1: 1105.426197052002


Processed prompts: 100%|██████████| 112/112 [06:03<00:00,  3.25s/it]


2: 363.5602478981018


Processed prompts: 100%|██████████| 112/112 [04:42<00:00,  2.52s/it]


...

8: 279.91154885292053


Processed prompts: 100%|██████████| 112/112 [05:07<00:00,  2.75s/it]


16: 307.66985177993774


Processed prompts: 100%|██████████| 112/112 [03:46<00:00,  2.02s/it]

32: 226.77632331848145


## 예제 8.6. 온라인 서빙을 위한 vLLM API 서버 실행

### 안내
모델을 여러 번 GPU에 올리기 때문에 CUDA out of memory 에러가 발생할 수 있습니다. 그런 경우 구글 코랩의 런타임 > 세션 다시 시작 후 예제 코드를 실행해주세요.
예제 실행에 데이터셋이 필요한 경우 예제 8.1의 코드를 실행해주세요.

In [2]:
!python -m vllm.entrypoints.openai.api_server \
--model shangrilar/yi-ko-6b-text2sql --host 127.0.0.1 --port 8888 --max-model-len 1024

INFO 04-19 13:01:59 api_server.py:151] vLLM API server version 0.4.1
INFO 04-19 13:01:59 api_server.py:152] args: Namespace(host='127.0.0.1', port=8888, uvicorn_log_level='info', allow_credentials=False, allowed_origins=['*'], allowed_methods=['*'], allowed_headers=['*'], api_key=None, served_model_name=None, lora_modules=None, chat_template=None, response_role='assistant', ssl_keyfile=None, ssl_certfile=None, ssl_ca_certs=None, ssl_cert_reqs=0, root_path=None, middleware=[], model='shangrilar/yi-ko-6b-text2sql', tokenizer=None, skip_tokenizer_init=False, revision=None, code_revision=None, tokenizer_revision=None, tokenizer_mode='auto', trust_remote_code=False, download_dir=None, load_format='auto', dtype='auto', kv_cache_dtype='auto', quantization_param_path=None, max_model_len=1024, guided_decoding_backend='outlines', worker_use_ray=False, pipeline_parallel_size=1, tensor_parallel_size=1, max_parallel_loading_workers=None, ray_workers_use_nsight=False, block_size=16, enable_prefix_ca

## 예제 8.7. 백그라운드에서 vLLM API 서버 실행하기

In [3]:
!nohup python -m vllm.entrypoints.openai.api_server \
--model shangrilar/yi-ko-6b-text2sql --host 127.0.0.1 --port 8888 --max-model-len 512 &

OSError: Background processes not supported.

## 예제 8.8. API 서버 실행 확인

In [5]:
# v1/models: 모델 정보를 제공하도록 구현된 경로
!curl http://localhost:8888/v1/models

{"object":"list","data":[{"id":"shangrilar/yi-ko-6b-text2sql","object":"model","created":1745153042,"owned_by":"vllm","root":"shangrilar/yi-ko-6b-text2sql","parent":null,"permission":[{"id":"modelperm-5242c11132054b4b83f2f0e0293f3fb4","object":"model_permission","created":1745153042,"allow_create_engine":false,"allow_sampling":true,"allow_logprobs":true,"allow_search_indices":false,"allow_view":true,"allow_fine_tuning":false,"organization":"*","group":null,"is_blocking":false}]}]}

## 예제 8.9. API 요청

In [5]:
import json

json_data = json.dumps(
    {"model": "shangrilar/yi-ko-6b-text2sql",
      "prompt": dataset.loc[0, "prompt"],
      "max_tokens": 128,
      "temperature": 1}
    )
# v1/completions: LLM 텍스트 생성 기능이 구현된 경로
!curl http://localhost:8888/v1/completions \
    -H "Content-Type: application/json" \
    -d '{json_data}'

{"id":"cmpl-efbfe28d7238472cb1adba91122d782f","object":"text_completion","created":1745139912,"model":"shangrilar/yi-ko-6b-text2sql","choices":[{"index":0,"text":"SELECT reward_items, SUM(reward_experience) FROM quests GROUP BY reward_items;","logprobs":null,"finish_reason":"stop","stop_reason":null}],"usage":{"prompt_tokens":127,"total_tokens":149,"completion_tokens":22}}

## 예제 8.10. OpenAI 클라이언트를 사용한 API 요청

In [2]:
%pip install openai==1.25.2 httpx==0.27.0 pydantic==2.6.4

  Using cached httpx-0.27.0-py3-none-any.whl.metadata (7.2 kB)
Using cached httpx-0.27.0-py3-none-any.whl (75 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 47.8 MB/s eta 0:00:00
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.33.1
    Uninstalling pydantic_core-2.33.1:
      Successfully uninstalled pydantic_core-2.33.1
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.11.3
    Uninstalling pydantic-2.11.3:
      Successfully uninstalled pydantic-2.11.3
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1
Note: you may need to restart the kernel to use updated packages.


In [6]:
from openai import OpenAI

# 로컬에서 실행 중인 vLLM 서버에 요청을 보내기 때문에 API 키는 EMPTY로 설정
openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8888/v1"
client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)
completion = client.completions.create(model="shangrilar/yi-ko-6b-text2sql",
                                 prompt=dataset.loc[0, 'prompt'], max_tokens=128)
print("생성 결과:", completion.choices[0].text)

생성 결과: SELECT reward_items, SUM(reward_experience) FROM quests GROUP BY reward_items;
